## 載入&重設定必要物件

In [ ]:
!pip install tensorflow-gpu==1.13.1
!pip install conlleval==0.1
#!pip install jieba==0.39
!pip install git+https://github.com/APCLab/jieba-tw.git
!pip install numpy==1.17.3
!pip install requests==2.22.0
!pip install tornado==6.0.3


In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
!pip install tensorflow==1.13.1

In [ ]:
!pip install keras==2.2.0

## 與雲端做連接

In [ ]:
# 如果出現了長條型的可輸入單元格 代表你曾經 授權過了 可以直接跳到下一格
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [ ]:
# 指定Google Drive雲端硬盤的根目錄，名為drive
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [ ]:
import os
os.chdir("drive")
if("Colab Notebooks" in os.listdir()):
  os.chdir("Colab Notebooks")
else:
  os.mkdir("Colab Notebooks")
  os.chdir("Colab Notebooks")

In [ ]:
!ls

105021007_homework4.ipynb  Ex8_1_Stock_Price_Prediction.ipynb
bilstm4ner		   NER-Ensemble_test02.ipynb
bilstm_ner_tf		   NerModel_test01.ipynb
CNN_minst_test.ipynb	   resemable-learning_test.ipynb
Ensemble_test.ipynb	   titanic
Ex6-1-MNIST.ipynb	   Titanic_UseEnsemble_test.ipynb
Ex6-2-Fashion-MNIST.ipynb  Untitled0.ipynb
Ex7-1-CNN.ipynb


## 主要內容

In [ ]:
# 数据读取与计算
import pandas as  pd
import matplotlib.pyplot as plt
import numpy as np

# 数据预处理与模型选择
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report
import itertools

# 随机森林与SVM
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#　%tensorflow_version 1.x
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

1.13.1


In [ ]:
#載入bilstm4ner的模型，因為有"bilstm_3000","bilstm_4000","bilstm_5000"這三種模型，不知道要選哪種，因此選個數字最大的
from keras.models import load_model

# !pip install git+https://www.github.com/keras-team/keras-contrib.git   ("""No module named 'keras_contrib'"""解決辦法)
# tensorflow指定1.13.1 keras指定2.2.0 keras-contrib指定2.0.8 版本其中一個錯誤會無法讀入模型
from keras_contrib.layers.crf import CRF, crf_loss, crf_viterbi_accuracy
model_path = 'bilstm4ner/model/bilstm_5000.h5'
modelPart1 = load_model(model_path, custom_objects={"CRF": CRF, 'crf_loss': crf_loss,
                          'crf_viterbi_accuracy': crf_viterbi_accuracy})
for layer in modelPart1.layers:
			layer.trainable = False
#new_model = tf.keras.models.load_model('C:/Users/danie/NER-Ensemble/bilstm_ner_tf/data_path_save/1521112368/checkpoints')
# model = create_model()
# model.load_weights('C:/Users/danie/NER-Ensemble/bilstm_ner_tf/data_path_save/1521112368/checkpoints')
# model.load_weights('bilstm_ner_tf/data_path_save/1521112368/checkpoints/model-31680.data-00000-of-00001')


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [ ]:
modelPart1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
masking_1 (Masking)          (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          1965600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300, 256)          439296    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 256)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 300, 256)          394240    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300, 256)          0         
__________

In [ ]:
model_path = 'bilstm4ner/model/bilstm_3000.h5'
modelPart2 = load_model(model_path, custom_objects={"CRF": CRF, 'crf_loss': crf_loss,
                          'crf_viterbi_accuracy': crf_viterbi_accuracy})
for layer in modelPart2.layers:
			layer.trainable = False
modelPart2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
masking_1 (Masking)          (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          1965600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300, 256)          439296    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 256)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 300, 256)          394240    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300, 256)          0         
__________

In [ ]:
new_model = keras.models.load_model('bilstm_ner_tf/data_path_save/1521112368/checkpoints')
#modelPart1.load_weights('bilstm_ner_tf/data_path_save/1521112368/checkpoints/model-31680.data-00000-of-00001')

OSError: ignored

In [ ]:
modelsEnsemble = [modelPart1, modelPart2]

In [ ]:
###暫時先用bilstm_3000和5000這兩個模型做測試

import pickle
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingCVClassifier
lr = LogisticRegression(n_jobs=-1, C=8)  # meta classifier
sclf = StackingCVClassifier(classifiers=[modelPart1, modelPart2], 
               #use_probas=True,
               #average_probas=False,
               meta_classifier=lr)

#sclf.fit(X, y)

## 參考code

In [ ]:
def ensemble(modelsEnsemble, model_input):


    outputs = [model.outputs[0] for model in modelsEmsemble]

    y = Average()(outputs)


    model = Model(model_input, y, name='ensemble')


    return model

ensemble_model = ensemble(modelsEnsemble, model_input)

In [ ]:
# -*- coding: utf-8 -*-

import pickle
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingCVClassifier

with open('../data/training_df.pkl', 'rb') as f:
    df = pickle.load(f)
with open(r'../data/selected_feat_names.pkl', 'rb') as f:
    selected_feat_names = pickle.load(f)
print("data loaded")

# train on full data set
y = df["attack_type"].values
X = df[selected_feat_names].values

xgb = XGBClassifier(learning_rate =0.5,n_estimators=300,max_depth=5,gamma=0,subsample=0.8,)
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=35, criterion="entropy")
etc = ExtraTreesClassifier(n_jobs=-1, n_estimators=5, criterion="entropy")
lr = LogisticRegression(n_jobs=-1, C=8)  # meta classifier

sclf = StackingCVClassifier(classifiers=[xgb, rfc, etc], meta_classifier=lr, use_probas=True, n_folds=3, verbose=3)

sclf.fit(X, y)
print("training finished")

# save model for later predicting
with open(r'../data/stacking.pkl', 'wb') as f:
    pickle.dump(sclf, f)
print("model dumped")

In [ ]:
# https://www.itread01.com/content/1542136503.html
# https://www.itread01.com/content/1548189913.html
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
                          use_probas=True,
                          average_probas=False,
                          meta_classifier=lr)

# 引數：

# classifiers : 基分類器，陣列形式，[cl1, cl2, cl3]. 每個基分類器的屬性被儲存在類屬性 self.clfs_.
# meta_classifier : 目標分類器，即將前面分類器合起來的分類器
# use_probas : bool (default: False) ，如果設定為True， 那麼目標分類器的輸入就是前面分類輸出的類別概率值而不是類別標籤
# average_probas : bool (default: False)，用來設定上一個引數當使用概率值輸出的時候是否使用平均值。
# verbose : int, optional (default=0)。用來控制使用過程中的日誌輸出，當 verbose = 0時，什麼也不輸出， verbose = 1，輸出迴歸器的序號和名字。verbose = 2，輸出詳細的引數資訊。verbose > 2, 自動將verbose設定為小於2的，verbose -2.
# use_features_in_secondary : bool (default: False). 如果設定為True，那麼最終的目標分類器就被基分類器產生的資料和最初的資料集同時訓練。如果設定為False，最終的分類器只會使用基分類器產生的資料訓練。

# 屬性：
# clfs_ : 每個基分類器的屬性，list, shape 為 [n_classifiers]。
# meta_clf_ : 最終目標分類器的屬性

# 方法：

# fit(X, y)
# fit_transform(X, y=None, fit_params)
# get_params(deep=True)，如果是使用sklearn的GridSearch方法，那麼返回分類器的各項引數。
# predict(X)
# predict_proba(X)
# score(X, y, sample_weight=None)， 對於給定資料集和給定label，返回評價accuracy
# set_params(params)，設定分類器的引數，params的設定方法和sklearn的格式一樣

In [ ]:
#https://ithelp.ithome.com.tw/articles/10187452
import numpy as np
import pandas as pd
from sklearn import cross_validation, ensemble, preprocessing, metrics

# 載入資料
url = "https://storage.googleapis.com/2017_ithome_ironman/data/kaggle_titanic_train.csv"
titanic_train = pd.read_csv(url)

# 填補遺漏值
age_median = np.nanmedian(titanic_train["Age"])
new_Age = np.where(titanic_train["Age"].isnull(), age_median, titanic_train["Age"])
titanic_train["Age"] = new_Age

# 創造 dummy variables
label_encoder = preprocessing.LabelEncoder()
encoded_Sex = label_encoder.fit_transform(titanic_train["Sex"])

# 建立訓練與測試資料
titanic_X = pd.DataFrame([titanic_train["Pclass"],
                         encoded_Sex,
                         titanic_train["Age"]
]).T
titanic_y = titanic_train["Survived"]
train_X, test_X, train_y, test_y = cross_validation.train_test_split(titanic_X, titanic_y, test_size = 0.3)

# 建立 bagging 模型
bag = ensemble.BaggingClassifier(n_estimators = 100)
bag_fit = bag.fit(train_X, train_y)

# 預測
test_y_predicted = bag.predict(test_X)

# 績效
accuracy = metrics.accuracy_score(test_y, test_y_predicted)
print(accuracy)